In [1]:
import src._generate as generate
import src._hammy as hammy
import src._tdp as tdp
import pandas as pd
import numpy as np
import plotly.express as px
import glob

# Generate data

***Put something in to clear out the data folder if there is already data there (and we are generating new stuff)

In [143]:
generate.generate_data(n_trajectories=200, state_means=[0,.5,1])

# Load our data

In [144]:
values = []

fnames = glob.glob('data/Trace*.csv')
for f in fnames:
    df = pd.read_csv(f)
    values.append(df.iloc[:,-1].values)

# Fit the data using HMM

***Have to put something in to prevent finding states right next to each other in FRET space.  If it does then the transitions will bounce between these two values and ruin the transition times.

In [145]:
predictions = hammy.fit_all(values, n_guessed_components=4, minimum_gap=0.2)

Model is not converging.  Current: 378.9276715214264 is not greater than 378.94155488601353. Delta is -0.01388336458711592
Model is not converging.  Current: 364.81840185849626 is not greater than 364.819185078906. Delta is -0.0007832204097439899
/Users/smc/miniconda3/envs/hammy/lib/python3.10/site-packages/hmmlearn/hmm.py:340: RuntimeWarning:

invalid value encountered in divide

Model is not converging.  Current: 359.7412824916995 is not greater than 359.74265418947084. Delta is -0.001371697771332947
/Users/smc/miniconda3/envs/hammy/lib/python3.10/site-packages/hmmlearn/hmm.py:374: RuntimeWarning:

overflow encountered in divide

/Users/smc/miniconda3/envs/hammy/lib/python3.10/site-packages/hmmlearn/hmm.py:340: RuntimeWarning:

invalid value encountered in divide

Some rows of transmat_ have zero sum because no transition from the state was ever observed.
/Users/smc/miniconda3/envs/hammy/lib/python3.10/site-packages/hmmlearn/hmm.py:340: RuntimeWarning:

invalid value encountered in d

Let's look at the first trace to see how it did

In [150]:
px.line(y=[values[0], predictions[0]])

In [151]:
np.unique(predictions[0])

array([0.00577575, 0.51790472, 1.00759551])

# Transition Density Plot

In [152]:
transitions, lengths = tdp.accumulate_transitions(predictions, minimum_separation=0.15)

In [153]:
tdf = pd.DataFrame(data={'source':transitions[:,0], 'sink':transitions[:,1]})
f = px.density_heatmap(tdf, x='source', y='sink', nbinsx=100, nbinsy=100, width=800, height=700)
f.write_html('heatmap_5000.html')
f

# Testing Gaussian Mixture Models

In [154]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from matplotlib.colors import LogNorm

In [155]:
def get_distribution(center_x, center_y, num_samples=100):
    C = np.array([[.02, 0], [.02, .03]])
    samples = np.random.randn(num_samples, 2)
    t = np.dot(samples, C) + [center_x,center_y]
    return t

    

In [156]:
states = [.1,.4, .8]
t = []
for x in states:
    for y in states:
        t.append(get_distribution(x,y, np.random.randint(20,1000)))
t = np.concatenate(t)
px.scatter(x=t[:,0], y=t[:,1], range_x=[-.1,1.1], range_y=[-.1, 1.1])

In [158]:
px.scatter(x=transitions[:,0], y=transitions[:,1], range_x=[-.1,1.1], range_y=[-.1, 1.1])

In [177]:
aics = []
my_range = np.arange(1,20)
for idx in my_range:
    gmm = GaussianMixture(n_components=idx)
    gmm.fit(transitions)
    subset = np.random.choice(np.arange(0,len(transitions)), size=100)
    aics.append(gmm.bic(transitions[subset]))
px.scatter(x=my_range, y=np.array(aics))

In [178]:
gmm = GaussianMixture(n_components=7)
gmm.fit(transitions)
aics.append(gmm.bic(transitions))
centers = gmm.means_
px.scatter(x=centers[:,0], y=centers[:,1])

In [167]:
gmm.bic?

Signature: gmm.bic(X)
Docstring:
Bayesian information criterion for the current model on the input X.

You can refer to this :ref:`mathematical section <aic_bic>` for more
details regarding the formulation of the BIC used.

Parameters
----------
X : array of shape (n_samples, n_dimensions)
    The input samples.

Returns
-------
bic : float
    The lower the better.
File:      ~/miniconda3/envs/hammy/lib/python3.10/site-packages/sklearn/mixture/_gaussian_mixture.py
Type:      method

# Playing with aggregating traces

SHould not really use this as the FRET values can wander between traces, not sure how well this would handle:  probably if a state is infrequently accessed it will split into two states a single state that IS frequently populated.

In [8]:
model, predictions = hammy.find_best_fit_combined(values, 4)

In [9]:
np.unique(predictions)

array([0.00443078, 0.00731405, 0.51096101, 1.00643805])

In [10]:
px.line(y=[np.array(values).flatten(), predictions])

In [11]:
fixed = hammy.cleanup_predictions(predictions, minimum_gap=0.1)

In [12]:
np.unique(fixed)

array([0.00443078, 0.51096101, 1.00643805])